In [ ]:
cd ../Python/

In [ ]:
import readPBNData.description as rd
import readPBNData.images as ri

In [ ]:
csvs, fileLike = rd.openZip('../Data/train_info.csv.zip')
lines, head = rd.readCSV(fileLike[0])

In [ ]:
cols = rd.columns(lines,head)
cols.keys()

In [ ]:
# find 10 artists with most paintings in data set
artistTable = rd.table(cols['artist'])
artistTable[:5]

In [ ]:
# images in train_1.zip and train_2.zip file
train_1_names = rd.imagesInZip('../Data/train_1.zip')
train_2_names = rd.imagesInZip('../Data/train_2.zip')

In [ ]:
# number of paintings by artist in train_1 and train_2
# distinct artists (slowish)
distinctArtists = [a[0] for a in artistTable]
paintingsIn12 = []
for artist in distinctArtists:
    paintingsIn12.append(len(rd.sameArtist(artist, cols, imageList=train_1_names))
                        +len(rd.sameArtist(artist, cols, imageList=train_2_names)))
artistTable12 = sorted(zip(distinctArtists, paintingsIn12), 
                       key=lambda item: item[1], reverse=True)
artistTable12[:10]

In [ ]:
# lists of paintings by the 10 "leading" artists in train_1 and train_2 
leaders = [a[0] for a in artistTable12[:10]]
portfolios1 = []
portfolios2 = []
for artist in leaders:
    portfolios1.append(rd.sameArtist(artist, cols, imageList=train_1_names))
    portfolios2.append(rd.sameArtist(artist, cols, imageList=train_2_names))
portfolios = zip(leaders,portfolios1,portfolios2)

In [ ]:
# create miniatures of the paintings by leading artists and save them to disk
miniportfolios = []
for portfolio in portfolios:
    minifiles_1 = ri.miniatures('../Data/train_1.zip', \
                                portfolio[1],prefix='train_1',size=(100,100))
    minifiles_2 = ri.miniatures('../Data/train_2.zip', \
                                portfolio[2],prefix='train_2',size=(100,100))
    miniportfolios.append((portfolio[0],minifiles_1,minifiles_2))

In [ ]:
# create cutouts of the paintings by leading artists and save them to disk
cutoutportfolios = []
for portfolio in portfolios:
    cutoutfiles_1 = ri.cutouts('../Data/train_1.zip', \
                               portfolio[1],prefix='train_1',size=(100,100))
    cutoutfiles_2 = ri.cutouts('../Data/train_2.zip', \
                               portfolio[2],prefix='train_2',size=(100,100))
    cutoutportfolios.append((portfolio[0],cutoutfiles_1,cutoutfiles_2))

In [ ]:
# prepare 1000 pairs of paintings by same artist 
# and 1000 pairs by different artists
import random
npairs = 1000
randomSeed = 666
random.seed(randomSeed)
portfolios_uni = [(artist, p1 + p2) for (artist,p1,p2) in portfolios]
pairs = []
for isample in xrange(npairs):
    # randomly choose an artist
    portfolio = random.choice(portfolios_uni)
    # randomly choose two paintings
    paintings = random.sample(portfolio[1],2)
    pairs.append([paintings[0],paintings[1],1])
for isample in xrange(npairs):
    # randomly choose two artists
    portfoliopair = random.sample(portfolios_uni,2)
    # randomly choose a painting from each artist
    paintings = random.choice(portfoliopair[0][1]), random.choice(portfoliopair[1][1])
    pairs.append([paintings[0],paintings[1],0])

In [ ]:
pairs[:5]